<a href="https://colab.research.google.com/github/srijan-singh/FaceDetect/blob/main/API_Face_Detect_V1_Base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastapi nest-asyncio pyngrok uvicorn
!pip install face_recognition
!pip install aiofiles
!pip install python-multipart

     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 747kB 39.6MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
     |████████████████████████████████| 61kB 8.8MB/s 
     |████████████████████████████████| 10.1MB 46.4MB/s 
     |████████████████████████████████| 92kB 12.6MB/s 
     |████████████████████████████████| 61kB 8.5MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.0.5-cp37-none-any.whl size=19246 sha256=3843958c6ebf35c50ef6a5d9c1077112b9c33f9ce6fd4920da2a5de5c288564b
  Stored in directory: /root/.cache/pip/wheels/0c/13/64/5ebbcc22eaf53fdf5766b397c1fb17c83f5775fdccf0ea1b88
Successfully built pyngrok
  Found existing installation: click 8.0.0
    Uninstalling click-8.0.0:
      Successfully uninstalled click-8.0.0
     |████████████████████████████████| 100.2MB 30kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566173 sha256=f3ea8969a31394bc0d8ee0fd59bc2b

In [1]:
#@title Class Face Detect
import face_recognition
import imageio
import cv2
import numpy as np


class faceDetect:

  def __init__ (self, user_image, user_label):

    face_1 = face_recognition.load_image_file(user_image)
    face_1_encoding = face_recognition.face_encodings(face_1)[0]

    self.known_face_encodings = [face_1_encoding]

    self.known_face_names = [user_label]

  def predict(self, pred_user_image):

    file_name = pred_user_image

    unknown_image = face_recognition.load_image_file(file_name)
    unknown_image_to_draw = cv2.imread(file_name)

    face_locations = face_recognition.face_locations(unknown_image)
    face_encodings = face_recognition.face_encodings(unknown_image, face_locations)

    for (top,right, bottom, left), face_encoding in zip(face_locations, face_encodings):

      matches = face_recognition.compare_faces(self.known_face_encodings, face_encoding)

      name = "Unknown"

      face_distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)
      best_match_index = np.argmin(face_distances)
      
      if matches[best_match_index]:
        
        name = self.known_face_names[best_match_index]
      cv2.rectangle(unknown_image_to_draw, (left, top), (right, bottom),(0,255,0),3)
      cv2.putText(unknown_image_to_draw,name, (left, top-20), cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2, cv2.LINE_AA)
  
    return cv2.imwrite("/content/pred_image.png", unknown_image_to_draw)

In [3]:
#@title API
#Libraries : Only for Colab (To Get Public Link)
import nest_asyncio
from pyngrok import ngrok
from pydantic import BaseModel

# Librarires : Must
import uvicorn
import shutil
from fastapi import FastAPI, UploadFile, File, Form
from fastapi.responses import FileResponse


app = FastAPI()

data_file = "upload_image.png"
target_file = "target_image.png"
data_label = ["Default"]

@app.get("/")
def index():
    return {"Yo":"LO"}


@app.post("/upload/data_file")
async def upload(file: UploadFile = File(...), username:str = Form(...)):
    
    data_label.append(username)

    with open(data_file, "wb") as buffer:
        shutil.copyfileobj(file.file, buffer)

    return {"filename": file.filename}


@app.post("/upload/target_file")
async def Target_Image(file: UploadFile = File(...)):

    with open(target_file, "wb") as buffer:
        shutil.copyfileobj(file.file, buffer)

    return {"filename": file.filename}


@app.get("/result")
def Prediction():
  model = faceDetect(data_file, data_label[-1])
  model.predict(target_file)
  return FileResponse("pred_image.png")



if __name__ == "__main__":
  ngrok_tunnel = ngrok.connect(8000)
  print('Public URL:', ngrok_tunnel.public_url)
  nest_asyncio.apply()
  uvicorn.run(app, port=8000)

INFO:     Started server process [402]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: http://bd8e04198f93.ngrok.io
INFO:     106.219.50.11:0 - "GET / HTTP/1.1" 200 OK
INFO:     106.219.50.11:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     49.44.83.36:0 - "GET / HTTP/1.1" 200 OK
INFO:     106.219.50.11:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     106.219.50.11:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     106.219.50.11:0 - "POST /upload/data_file HTTP/1.1" 200 OK
INFO:     106.219.50.11:0 - "POST /upload/target_file HTTP/1.1" 200 OK
INFO:     106.219.50.11:0 - "GET /result HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [402]
